<a href="https://colab.research.google.com/github/SevilAnna/GraduationProject/blob/main/EMU403_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>EMÜ 403 - Graduation Project</h1>

---


<font color='red'> 🔴🔴 Kızlar 🔴🔴</font> 
</br></br>Kod üzerinde çalışacaksanız:
- Öncelikle kodun en son versiyonunun <a href="https://github.com/SevilAnna/GraduationProject">Github</a>'da olduğundan emin olun. 
- Drive'daki "Düzenlenmiş_Dişli_Parçaları" Excelini kendi bilgisayarınıza indirin. 
- Veriyi çekmek için aşağıdaki en az ilk 3 kutudaki kodu çalıştırın.
- Github'a kaydederken "main branch"e değil, başka bir yan branch'e kaydedin.
</br></br>- Sevil</br></br>

## Importing Libraries & the Excel File

### Importing libraries for the project

In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import random
import copy
#!pip install geneticalgorithm
#import geneticalgorithm as ga
#from geneticalgorithm import geneticalgorithm as ga
print("done")

done


### Importing the Excel file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Düzenlenmiş_Dişli_Parçaları.xlsx to Düzenlenmiş_Dişli_Parçaları (1).xlsx


In [3]:
import io
df = pd.read_excel(io.BytesIO(uploaded["Düzenlenmiş_Dişli_Parçaları.xlsx"]), sheet_name='Anka', header=0)
del df['SÜRE (11 parça için)']
del df['1 tanesi için kaç tane gerek']
del df['PROSES']
del df['Process time']
pr_num = list(range(1,len(df)+1))
df["Process #"] = pr_num
df = df.rename(columns={'Makine':'Machine', 'İşlem':'Process', 'SETUP':'Setup', 
                    'SÜRE (1 parça için)':'Processing time', 'PARÇA':'Piece'})
df = df.reindex(columns=['Piece','Process #','Process','Machine','Setup','Processing time'])
df

,Piece,Process #,Process,Machine,Setup,Processing time
0,UM121T900201200 GEAR ENGINE Z35,1,Pre-Hardening Öncesi Torna,D13,0.0,1.000000
1,UM121T900201200 GEAR ENGINE Z35,2,Yıkama,D33,0.0,0.090909
2,UM121T900201200 GEAR ENGINE Z35,3,Ön Sertleştirme (Isıl İşlem),D31,0.0,0.545455
3,UM121T900201200 GEAR ENGINE Z35,4,Yıkama,D33,0.0,0.090909
4,UM121T900201200 GEAR ENGINE Z35,5,Temperleme (Isıl İşlem),D32,0.0,0.454545
...,...,...,...,...,...,...
315,UM121T900204300 GEAR ALTERNATOR Z13 -2,316,Yıkama,D12,0.0,0.181818
316,UM121T900204300 GEAR ALTERNATOR Z13 -2,317,Balans,D69,0.0,1.000000
317,UM121T900204300 GEAR ALTERNATOR Z13 -2,318,MPI,D65,0.0,0.454545
318,UM121T900204300 GEAR ALTERNATOR Z13 -2,319,Yıkama,D12,0.0,0.090909


## Chromosomes

In [4]:
m_list = np.unique(np.array(df["Machine"]))                         # Each machine needed
m_num_list = [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]    # How many there're of each machine (done manually now; to automate later)
print(len(m_list),m_list)
print(len(m_num_list),m_num_list)

17 ['662' 'D11' 'D12' 'D13' 'D15' 'D17' 'D27' 'D31' 'D32' 'D33' 'D41' 'D43'
 'D44' 'D47' 'D65' 'D69' 'Nital Etche, Hydrojen Embr.']
17 [1, 2, 1, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 1, 2, 1, 1]


In [5]:
# creating a dictionary with each machine (including duplicates)
all_machines = {}
n = 0
for m in m_list:
  m_num = m_num_list[n]
  if m_num > 0:
    all_machines[m] = []
    # Adding processes to the machines (but not to duplicates)
    for r in range(len(df["Machine"])):
      if m == df.iloc[r]["Machine"]:
        all_machines[m].append(r+1)
  # add duplicate machines without the processes
  if m_num > 1:
    m1 = m+"-1"
    all_machines[m1] = all_machines[m]
    all_machines.pop(m)
    if m_num > 1:
      m2 = m+"-2"
      all_machines[m2] = []
    if m_num > 2:
      m3 = m+"-3"
      all_machines[m3] = []
    if m_num > 3:
      m4 = m+"-4"
      all_machines[m4] = []   # later add more if there're 4+ of a certain machine ---> or just write a function for this 
  n += 1
   
for k in all_machines:
  print(k,"\t",all_machines[k])

662 	 [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320]
D11-1 	 [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311]
D11-2 	 []
D12 	 [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319]
D13-1 	 [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295]
D13-2 	 []
D13-3 	 []
D13-4 	 []
D15-1 	 [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313]
D15-2 	 []
D15-3 	 []
D17-1 	 [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310]
D17-2 	 []
D27-1 	 [21, 53, 85, 117, 149, 181, 213, 245, 277, 309]
D27-2 	 []
D27-3 	 []
D31-1 	 [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175, 195, 207, 227, 239, 259, 271, 29

notes for presentation:
- we can see that D12 could be a bottleneck for ANKA

In [6]:
machines = copy.deepcopy(all_machines)

# initialize genetic algorithm where each gene is a machine's schedule
num_genes = len(machines)
num_chro = 8
popu_size = (num_chro,num_genes)
popu = []


for i in range(num_chro):
  popu.append([machines])

# shuffle the order of processes in each machine/gene except for the first chromosome, and print results
new_popu = []
for i in range(len(popu)):
  new_popu.append([0])
first_one = True
for i in range(len(popu)):
  if first_one == False:
    for j in popu[i]:
      for k in j:
        temp_l = random.sample(copy.deepcopy(list(j[k])),len(j[k]))
        j[k] = copy.deepcopy(temp_l)
  first_one = False
  new_popu[i] = copy.deepcopy(popu[i])

for c in new_popu:
  print(c)

[{'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11-1': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D11-2': [], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13-1': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D13-2': [], 'D13-3': [], 'D13-4': [], 'D15-1': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D15-2': [], 'D15-3': [], 'D17-1': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D17-2': [], 'D27-1': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D27-2': [], 'D27-3': [], 'D31-1': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 17

## Fitness function

In [7]:
# setup, processing & total times for each process (from 1 to 320)
pr_times = list(df["Processing time"])
stp_times = list(df["Setup"])
t_times = []
for i in range(len(pr_times)):
  t_times.append(pr_times[i] + stp_times[i])
print(pr_times)
print(stp_times)
print(t_times)

[1.0, 0.09090909091, 0.5454545455, 0.09090909091, 0.4545454545, 0.4545454545, 1.636363636, 0.09090909091, 0.09090909091, 0.4545454545, 2.0, 0.4545454545, 0.1818181818, 0.4545454545, 0.8181818182, 0.09090909091, 0.2727272727, 0.5454545455, 0.4545454545, 0.4545454545, 1.0, 0.4545454545, 2.0, 0.1818181818, 0.4545454545, 0.4545454545, 1.272727273, 0.1818181818, 2.0, 0.4545454545, 0.1818181818, 1.0, 0.75, 0.09090909091, 0.5454545455, 0.09090909091, 0.4545454545, 0.4545454545, 1.227272727, 0.06818181818, 0.06818181818, 0.3409090909, 1.5, 0.3409090909, 0.1363636364, 0.3409090909, 0.8181818182, 0.09090909091, 0.2727272727, 0.5454545455, 0.4545454545, 0.3409090909, 0.75, 0.3409090909, 1.5, 0.1363636364, 0.3409090909, 0.3409090909, 1.272727273, 0.1363636364, 1.5, 0.4545454545, 0.1363636364, 0.75, 0.75, 0.09090909091, 0.5454545455, 0.09090909091, 0.4545454545, 0.4545454545, 1.227272727, 0.06818181818, 0.06818181818, 0.3409090909, 1.5, 0.3409090909, 0.1363636364, 0.3409090909, 0.8181818182, 0.0909

In [8]:
# function to write setup, processing & total times for each process in a chromosome
def write_times(m_time):
  m_pr_time = copy.deepcopy(m_time)
  m_stp_time = copy.deepcopy(m_time)
  m_tot_time = copy.deepcopy(m_time)
  for m in m_time:
    for i in range(len(m_time[m])):
      for j in range(len(pr_times)):
        if m_time[m][i] == j+1:
          m_pr_time[m][i] = pr_times[j]
          m_stp_time[m][i] = stp_times[j]
          m_tot_time[m][i] = t_times[j]
  return m_pr_time, m_stp_time, m_tot_time

pr_time, stp_time, t_time = write_times(all_machines)
print("process:",pr_time)
print("setups:",stp_time)
print("total:",t_time)

process: {'662': [0.4545454545, 1.0, 0.3409090909, 0.75, 0.3409090909, 0.75, 0.4545454545, 1.0, 0.4545454545, 1.0, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5], 'D11-1': [2.0, 2.0, 1.5, 1.5, 1.5, 1.5, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'D11-2': [], 'D12': [0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091], 'D13-1': [1.0, 1.636363636, 0.75, 1.227272727, 0.75, 1.227272727, 1.0, 1.63636363

In [18]:
# for ease
print(all_machines)
print(all_machines["662"][0])

{'662': [26, 32, 58, 64, 90, 96, 122, 128, 154, 160, 186, 192, 218, 224, 250, 256, 282, 288, 314, 320], 'D11-1': [11, 23, 43, 55, 75, 87, 107, 119, 139, 151, 171, 183, 203, 215, 235, 247, 267, 279, 299, 311], 'D11-2': [], 'D12': [13, 24, 28, 31, 45, 56, 60, 63, 77, 88, 92, 95, 109, 120, 124, 127, 141, 152, 156, 159, 173, 184, 188, 191, 205, 216, 220, 223, 237, 248, 252, 255, 269, 280, 284, 287, 301, 312, 316, 319], 'D13-1': [1, 7, 33, 39, 65, 71, 97, 103, 129, 135, 161, 167, 193, 199, 225, 231, 257, 263, 289, 295], 'D13-2': [], 'D13-3': [], 'D13-4': [], 'D15-1': [14, 20, 25, 46, 52, 57, 78, 84, 89, 110, 116, 121, 142, 148, 153, 174, 180, 185, 206, 212, 217, 238, 244, 249, 270, 276, 281, 302, 308, 313], 'D15-2': [], 'D15-3': [], 'D17-1': [10, 22, 42, 54, 74, 86, 106, 118, 138, 150, 170, 182, 202, 214, 234, 246, 266, 278, 298, 310], 'D17-2': [], 'D27-1': [21, 53, 85, 117, 149, 181, 213, 245, 277, 309], 'D27-2': [], 'D27-3': [], 'D31-1': [3, 15, 35, 47, 67, 79, 99, 111, 131, 143, 163, 175

In [22]:
# function to write the start & finishing times for each process
def write_start_finish(m_time):
  m_S_time = copy.deepcopy(t_time)
  m_F_time = copy.deepcopy(t_time)
  for m in m_time:                    # each machine's list
    for i in range(len(m_time[m])):   # 0 to length of each machine (each process in a machine)
      for j in range(len(t_times)):    # 0 to 319 (every single process)
        if m_time[m][i] == j+1:       # if process in m_time = a process
          if i == 0:
            m_S_time[m][i] = 0            ### start first process at 0 for now
            m_F_time[m][i] = t_time[m][i]
          else:
            m_S_time[m][i] = m_S_time[m][i-1] + t_time[m][i-1]  # or m_F_time[m][i-1]
            m_F_time[m][i] = m_S_time[m][i] + t_time[m][i]
  return m_S_time, m_F_time

S_time, F_time = write_start_finish(all_machines)
print("t_time ",t_time)
print("start  ",S_time)
print("finish ",F_time)

t_time  {'662': [0.4545454545, 1.0, 0.3409090909, 0.75, 0.3409090909, 0.75, 0.4545454545, 1.0, 0.4545454545, 1.0, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5, 0.2272727273, 0.5], 'D11-1': [7.5, 7.5, 5.625, 5.625, 5.625, 5.625, 7.5, 7.5, 7.5, 7.5, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75, 3.75], 'D11-2': [], 'D12': [0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1363636364, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.1818181818, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091, 0.09090909091, 0.09090909091, 0.1818181818, 0.09090909091], 'D13-1': [1.0, 1.636363636, 0.75, 1.227272727, 0.75, 1.227272727

In [11]:
sdf

NameError: ignored

</br>      

---
   
</br>      

To do:
  - add sth to the genes that represent idle times & then shuffle & adjust idle time length
    - adjust the finish times accordinly
  - check the processes in order across machines & give partial score when testing fitness
    - cumulative times for each process in the chromosomes for fitness testing
    - will need to check predecessors


## Others

In [ ]:
a_list = np.unique(np.array(df["Piece"]))
num = df["Piece"].nunique(a_list[0])
u_num = [0]*num
for i in range(9):
  for item in df["Piece"]:
    if item == a_list[i]: 
      u_num[i] += 1
  print(u_num[i],"processes for piece",a_list[i])

In [ ]:
processes = []
for i in range(num):
  processes.append([])
#print(processes)

j = 0
k = 0
total = 0
for i in range(len(u_num)):
  total += u_num[i]
  while j < total:
    if j == total:
      k += 1
    #print(i,j,k,total,processes)
    #print(processes[i])
    processes[i].append(j+1)
    j += 1
  #print(total)

for i in processes:
  print(len(i),i)

### Beyza's part

In [ ]:

Srt_time =0

pr_times = list(df["Processing time"])
stp_times = list(df["Setup"])
a_list = np.unique(np.array(df["Piece"]))
num = df["Piece"].nunique(a_list[0])
print(a_list)
#print(Start_time)

processes = []
for i in range(num):
  processes.append([])
#print(processes)

j = 0
k = 0
total = 0
for i in range(len(u_num)):
  total += u_num[i]
  while j < total:
    if j == total:
      k += 1
    #print(i,j,k,total,processes)
    #print(processes[i])
    processes[i].append(j+1)
    j += 1
  #print(total)

k =0
Devam =0 # bunu denemek amaçlı koyuyom 

for i in processes:
  for j in range(len(i)):
    Devam += 1
    #print(Devam)
    Srt_time += pr_times[Devam]
    print("Start schedule for the" ,j,".","job",Srt_time)
